In [31]:
import os
import sys
import pandas as pd

import numpy as np
from datetime import datetime as dt
import calendar
import matplotlib.pyplot as plt

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import warnings
warnings.filterwarnings('ignore')

In [34]:
data_dir = '/Users/maoyabassiouni/Documents/DATA/Projects/OLNI'

# Load LT data

In [35]:
df_all = pd.read_csv(os.path.join(data_dir, 'model_inputs', 'df_model_all_LT.csv'))
df_all = df_all.drop([ 'Unnamed: 0', 'index'], axis=1)
df_all['Lat'] = np.round(df_all['Lat'], 4)
df_all['Lon'] = np.round(df_all['Lon'], 4)
df_all = df_all.drop(['CO2',], axis=1)

In [36]:
df_all

,Lat,Lon,n,ymin,ymax,count,yrange,sp,ERA5_rf_alpha_LTavg,ERA5_rf_lambda_LTavg,...,Td,T_GS,porosity,Ks,s_h,s_fc,fparl,elevation,forest_height,elev_pa
0,34.9294,32.9311,9.772500,2005,2011,4,6,121.0,0.004896,0.297595,...,45805.0,183.0,0.435,2.99808,0.178245,0.729896,0.9,1340.0,20,86699.589354
1,34.9294,32.9311,8.696250,2005,2011,4,6,129.0,0.004896,0.297595,...,45805.0,183.0,0.435,2.99808,0.178245,0.729896,0.9,1340.0,20,86699.589354
2,34.9522,32.8306,7.951250,2005,2011,4,6,129.0,0.004896,0.297595,...,45987.0,183.0,0.435,2.99808,0.178245,0.729896,0.9,1425.0,0,85832.982620
3,35.0139,33.0500,11.116250,2005,2011,4,6,121.0,0.005832,0.189129,...,44726.0,184.0,0.420,0.54432,0.319021,0.841720,0.9,743.0,0,92987.997573
4,35.0200,32.6275,9.713750,2005,2011,4,6,121.0,0.006450,0.191424,...,46558.0,183.0,0.420,0.54432,0.319021,0.841720,0.9,757.0,20,92836.419952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,67.9994,24.2361,9.700758,1995,2015,11,20,118.0,0.002654,0.691699,...,38036.0,122.0,0.435,2.99808,0.178245,0.729896,0.9,345.0,15,97381.720048
992,68.9953,19.4275,12.715000,1995,1999,3,4,118.0,0.003270,0.937913,...,42031.0,92.0,0.435,2.99808,0.178245,0.729896,0.9,200.0,16,99023.609311
993,69.4542,30.0361,11.912000,1995,2003,5,8,134.0,0.002487,0.675428,...,46758.0,122.0,0.435,2.99808,0.178245,0.729896,0.9,33.0,13,100942.316272
994,69.5831,28.8981,12.803636,1995,2015,11,20,134.0,0.002544,0.676992,...,48358.0,122.0,NaN,NaN,NaN,NaN,0.9,105.0,14,100111.431748


# Join annual data

In [37]:
data_ERA5 = pd.read_csv(os.path.join(data_dir, 'grid_data_extract', 'era5_annual.csv'))
data_ERA5['Lat'] = np.round(data_ERA5['Lat'], 4)
data_ERA5['Lon'] = np.round(data_ERA5['Lon'], 4)
data_ERA5 = data_ERA5.groupby(['Lon', 'Lat', 'year']).mean()
data_ERA5 = data_ERA5.drop(['Unnamed: 0'], axis=1)

data_ERA5['tmp'] = data_ERA5['mean_2m_air_temperature']
data_ERA5['tmp_gs'] = data_ERA5['mean_2m_air_temperature_gs']
data_ERA5['tmp_gs_peak'] = data_ERA5['mean_2m_air_temperature_gs_peak']

data_ERA5['pre'] = data_ERA5['total_precipitation']
data_ERA5['pre_gs'] = data_ERA5['total_precipitation_gs']
data_ERA5['pre_gs_peak'] = data_ERA5['total_precipitation_gs_peak']

data_ERA5['pa'] = data_ERA5['surface_pressure']
data_ERA5['pa_gs'] = data_ERA5['surface_pressure_gs']
data_ERA5['pa_gs_peak'] = data_ERA5['surface_pressure_gs_peak']

data_ERA5 = data_ERA5.drop(['mean_2m_air_temperature', 'mean_2m_air_temperature_gs', 'mean_2m_air_temperature_gs_peak',
                           'total_precipitation', 'total_precipitation_gs', 'total_precipitation_gs_peak',
                           'surface_pressure', 'surface_pressure_gs', 'surface_pressure_gs_peak',], 
                           axis=1)

keys = data_ERA5.keys()
for kk in keys:
    data_ERA5['ERA5_%s_yi' % kk] = data_ERA5[kk]
    data_ERA5 = data_ERA5.drop(['%s' % kk,], axis=1)

data_ERA5 = data_ERA5.reset_index()
df_all = df_all.reset_index()
df_all = pd.merge(df_all, data_ERA5, on=['Lat', 'Lon', ], how='outer')
df_all = df_all.drop(['index'], axis=1)

print(len(data_ERA5.groupby(['Lon', 'Lat']).count().index))
print(len(data_ERA5.groupby(['Lon', 'Lat', 'year']).count().index)
      / len(data_ERA5.groupby(['Lon', 'Lat']).count().index))
print(len(df_all.groupby(['Lon', 'Lat', 'sp']).count().index))
print(keys)
df_all

903
27.0
996
Index(['rf_alpha', 'rf_lambda', 'rf_alpha_gs', 'rf_lambda_gs', 'tmp', 'tmp_gs',
       'tmp_gs_peak', 'pre', 'pre_gs', 'pre_gs_peak', 'pa', 'pa_gs',
       'pa_gs_peak'],
      dtype='object')


,Lat,Lon,n,ymin,ymax,count,yrange,sp,ERA5_rf_alpha_LTavg,ERA5_rf_lambda_LTavg,...,ERA5_rf_lambda_gs_yi,ERA5_tmp_yi,ERA5_tmp_gs_yi,ERA5_tmp_gs_peak_yi,ERA5_pre_yi,ERA5_pre_gs_yi,ERA5_pre_gs_peak_yi,ERA5_pa_yi,ERA5_pa_gs_yi,ERA5_pa_gs_peak_yi
0,34.9294,32.9311,9.772500,2005,2011,4,6,121.0,0.004896,0.297595,...,0.146084,17.196190,22.128936,23.532169,0.000853,0.000210,0.000130,94509.448081,94235.876580,94213.834635
1,34.9294,32.9311,9.772500,2005,2011,4,6,121.0,0.004896,0.297595,...,0.100125,16.882300,21.962726,24.026341,0.001853,0.000271,0.000015,94435.121126,94247.802852,94168.004167
2,34.9294,32.9311,9.772500,2005,2011,4,6,121.0,0.004896,0.297595,...,0.124708,15.813960,21.410113,22.685417,0.001415,0.000450,0.000198,94490.861574,94262.233649,94121.036979
3,34.9294,32.9311,9.772500,2005,2011,4,6,121.0,0.004896,0.297595,...,0.170516,17.038295,22.290895,24.086152,0.001019,0.000494,0.000194,94512.584782,94238.559469,94218.334115
4,34.9294,32.9311,9.772500,2005,2011,4,6,121.0,0.004896,0.297595,...,0.260296,17.898456,23.481357,24.066933,0.001963,0.000439,0.000018,94403.222282,94178.074112,94157.094531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26887,69.7389,26.9581,11.593125,1995,2009,8,14,134.0,0.002348,0.708140,...,1.051614,-0.886492,8.588691,10.840679,0.001692,0.002427,0.002491,97300.164062,97350.359695,97294.891381
26888,69.7389,26.9581,11.593125,1995,2009,8,14,134.0,0.002348,0.708140,...,0.719465,0.891957,11.871532,12.950098,0.001474,0.001844,0.003791,97236.037393,97821.666752,97697.735383
26889,69.7389,26.9581,11.593125,1995,2009,8,14,134.0,0.002348,0.708140,...,1.049658,0.284777,10.263237,14.523825,0.001473,0.002013,0.001136,97470.795826,97768.929495,98300.068548
26890,69.7389,26.9581,11.593125,1995,2009,8,14,134.0,0.002348,0.708140,...,0.994953,0.518596,9.587019,9.604426,0.001766,0.002282,0.001251,97015.403960,97631.932249,97538.934224


In [38]:
data_TC = pd.read_csv(os.path.join(data_dir, 'grid_data_extract', 'tc_annual.csv'))
data_TC['Lat'] = np.round(data_TC['Lat'], 4)
data_TC['Lon'] = np.round(data_TC['Lon'], 4)
data_TC = data_TC.groupby(['Lon', 'Lat', 'year']).mean()
data_TC = data_TC.drop(['Unnamed: 0'], axis=1)

data_TC['pre'] = data_TC['pr']
data_TC['pre_gs'] = data_TC['pr_gs']
data_TC['pre_gs_peak'] = data_TC['pr_gs_peak']
data_TC = data_TC.drop(['pr', 'pr_gs', 'pr_gs_peak'], axis=1)

data_TC['tmp'] = (data_TC['tmmn'] + data_TC['tmmx']) / 2
data_TC['tmp_gs'] = (data_TC['tmmn_gs'] + data_TC['tmmx_gs']) / 2
data_TC['tmp_gs_peak'] = (data_TC['tmmn_gs_peak'] + data_TC['tmmx_gs_peak']) / 2


data_TC['par'] = data_TC['srad'] * 10 ** (-6) * 2.04
data_TC['par_gs'] = data_TC['srad_gs'] * 10 ** (-6) * 2.04
data_TC['par_gs_peak'] = data_TC['srad_gs_peak'] * 10 ** (-6) * 2.04

keys = data_TC.keys()
for kk in keys:
    data_TC['TC_%s_yi' % kk] = data_TC[kk]
    data_TC = data_TC.drop(['%s' % kk,], axis=1)

data_TC = data_TC.reset_index()
df_all = df_all.reset_index()
df_all = pd.merge(df_all, data_TC, on=['Lat', 'Lon', 'year'], how='outer')
df_all = df_all.drop(['index'], axis=1)

print(len(data_TC.groupby(['Lon', 'Lat']).count().index))
print(len(df_all.groupby(['Lon', 'Lat', 'sp']).count().index))
print(keys)
df_all

903
996
Index(['srad', 'pet', 'aet', 'def', 'pdsi', 'ro', 'soil', 'tmmn', 'tmmx',
       'vap', 'vpd', 'vs', 'days_month', 'tmean', 'aet_gs', 'def_gs',
       'month_gs', 'pdsi_gs', 'pet_gs', 'ro_gs', 'soil_gs', 'srad_gs',
       'tmean_gs', 'tmmn_gs', 'tmmx_gs', 'vap_gs', 'vpd_gs', 'vs_gs',
       'aet_gs_peak', 'def_gs_peak', 'month_gs_peak', 'pdsi_gs_peak',
       'pet_gs_peak', 'ro_gs_peak', 'soil_gs_peak', 'srad_gs_peak',
       'tmean_gs_peak', 'tmmn_gs_peak', 'tmmx_gs_peak', 'vap_gs_peak',
       'vpd_gs_peak', 'vs_gs_peak', 'pre', 'pre_gs', 'pre_gs_peak', 'tmp',
       'tmp_gs', 'tmp_gs_peak', 'par', 'par_gs', 'par_gs_peak'],
      dtype='object')


,Lat,Lon,n,ymin,ymax,count,yrange,sp,ERA5_rf_alpha_LTavg,ERA5_rf_lambda_LTavg,...,TC_vs_gs_peak_yi,TC_pre_yi,TC_pre_gs_yi,TC_pre_gs_peak_yi,TC_tmp_yi,TC_tmp_gs_yi,TC_tmp_gs_peak_yi,TC_par_yi,TC_par_gs_yi,TC_par_gs_peak_yi
0,34.9294,32.9311,9.772500,2005,2011,4,6,121.0,0.004896,0.297595,...,2.48,0.001502,0.000643,0.000333,13.468182,17.766667,18.65,0.000445,0.000566,0.000633
1,34.9294,32.9311,8.696250,2005,2011,4,6,129.0,0.004896,0.297595,...,2.48,0.001502,0.000643,0.000333,13.468182,17.766667,18.65,0.000445,0.000566,0.000633
2,34.9294,32.9311,9.772500,2005,2011,4,6,121.0,0.004896,0.297595,...,1.85,0.001993,0.000395,0.000000,13.204167,17.675000,18.25,0.000417,0.000558,0.000622
3,34.9294,32.9311,8.696250,2005,2011,4,6,129.0,0.004896,0.297595,...,1.85,0.001993,0.000395,0.000000,13.204167,17.675000,18.25,0.000417,0.000558,0.000622
4,34.9294,32.9311,9.772500,2005,2011,4,6,121.0,0.004896,0.297595,...,2.30,0.003200,0.001614,0.001067,12.062500,17.375000,18.20,0.000418,0.000560,0.000615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26887,69.7389,26.9581,11.593125,1995,2009,8,14,134.0,0.002348,0.708140,...,3.13,0.001339,0.001967,0.001871,-0.654167,9.362500,12.15,0.000169,0.000286,0.000335
26888,69.7389,26.9581,11.593125,1995,2009,8,14,134.0,0.002348,0.708140,...,2.62,0.000978,0.001184,0.001968,-0.404167,11.875000,13.80,0.000188,0.000315,0.000366
26889,69.7389,26.9581,11.593125,1995,2009,8,14,134.0,0.002348,0.708140,...,2.22,0.001065,0.001528,0.001032,-0.287500,10.462500,15.00,0.000187,0.000330,0.000447
26890,69.7389,26.9581,11.593125,1995,2009,8,14,134.0,0.002348,0.708140,...,2.39,0.001127,0.001578,0.000871,-0.020833,10.062500,11.00,0.000165,0.000290,0.000375


In [39]:
df_co2 = pd.read_csv(os.path.join(data_dir, 'grid_data_extract', 'co2_annual.csv'))
df_co2 = df_co2.drop([ 'Unnamed: 0'], axis=1)
df_co2 = df_co2[(df_co2['year']>=1990) & (df_co2['year']<=2016)]

df_co2 = df_co2.reset_index()
df_all = pd.merge(df_co2, df_all, on=['year', ], how='outer')
df_all

df_all = df_all.drop(['index'], axis=1)
print(len(df_all.groupby(['Lon', 'Lat', 'sp']).count().index))
df_all['CO2'] = df_all['CO2'] * 10 ** -6
df_all

996


,year,CO2,Lat,Lon,n,ymin,ymax,count,yrange,sp,...,TC_vs_gs_peak_yi,TC_pre_yi,TC_pre_gs_yi,TC_pre_gs_peak_yi,TC_tmp_yi,TC_tmp_gs_yi,TC_tmp_gs_peak_yi,TC_par_yi,TC_par_gs_yi,TC_par_gs_peak_yi
0,1990,0.000354,34.9294,32.9311,9.772500,2005,2011,4,6,121.0,...,2.48,0.001502,0.000643,0.000333,13.468182,17.766667,18.65,0.000445,0.000566,0.000633
1,1990,0.000354,34.9294,32.9311,8.696250,2005,2011,4,6,129.0,...,2.48,0.001502,0.000643,0.000333,13.468182,17.766667,18.65,0.000445,0.000566,0.000633
2,1990,0.000354,34.9522,32.8306,7.951250,2005,2011,4,6,129.0,...,2.88,0.001351,0.000583,0.000300,14.090909,18.233333,19.15,0.000452,0.000575,0.000641
3,1990,0.000354,35.0139,33.0500,11.116250,2005,2011,4,6,121.0,...,2.82,0.000958,0.000632,0.000433,17.254545,19.383333,14.45,0.000470,0.000578,0.000512
4,1990,0.000354,35.0200,32.6275,9.713750,2005,2011,4,6,121.0,...,2.83,0.001255,0.000562,0.000333,14.309091,18.283333,19.40,0.000454,0.000577,0.000640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26887,2016,0.000404,67.9994,24.2361,9.700758,1995,2015,11,20,118.0,...,2.45,0.001705,0.003113,0.004194,0.466667,10.712500,14.80,0.000173,0.000273,0.000347
26888,2016,0.000404,68.9953,19.4275,12.715000,1995,1999,3,4,118.0,...,2.31,0.001479,0.002159,0.002613,2.654167,11.650000,13.95,0.000174,0.000323,0.000335
26889,2016,0.000404,69.4542,30.0361,11.912000,1995,2003,5,8,134.0,...,3.25,0.001578,0.002488,0.002387,1.625000,11.212500,14.85,0.000171,0.000285,0.000373
26890,2016,0.000404,69.5831,28.8981,12.803636,1995,2015,11,20,134.0,...,2.81,0.001449,0.002324,0.002355,1.354167,11.212500,15.00,0.000169,0.000277,0.000355


### add obs data

In [40]:
dfR_icp = pyreadr.read_r(os.path.join(data_dir, 'raw', 'foliarNutrientsEnv_R', 'icp_trends.Rdata'))
dfR_icp = pd.DataFrame(dfR_icp['icp.trends'])
dfR_icp[['lat', 'lon', 'year', 'code_tree_species', 'n']].dropna()
dfR_icp['Lat'] = np.round(dfR_icp['lat'], 4)
dfR_icp['Lon'] = np.round(dfR_icp['lon'], 4)
dfR_icp['sp'] = dfR_icp['code_tree_species']
dfR_icp['LNC'] = dfR_icp['n']
dfR_icp = dfR_icp[['Lat', 'Lon', 'year', 'sp', 'LNC']]
dfR_icp = dfR_icp[(dfR_icp['LNC']>0) & (dfR_icp['year']>=1990) & (dfR_icp['year']<=2016)]

In [41]:
df_locs = pd.read_csv(os.path.join(data_dir, 'sites', 'all_icp_sites.csv'))
df_locs['Lat'] = np.round(df_locs['lat'], 4)
df_locs['Lon'] = np.round(df_locs['lon'], 4)
df_locs['sp'] = np.round(df_locs['code_tree_species'], 0)
df_locs = df_locs[['Lat', 'Lon', 'sp']]
df_locs = [list(arr) for arr in df_locs[['Lat', 'Lon', 'sp']].values]
print(len(df_locs))
df_locs

996


[[34.9294, 32.9311, 121.0],
 [34.9294, 32.9311, 129.0],
 [34.9522, 32.8306, 129.0],
 [35.0139, 33.05, 121.0],
 [35.02, 32.6275, 121.0],
 [36.3706, -5.5783, 54.0],
 [37.0314, -3.015, 46.0],
 [37.1553, -6.7325, 131.0],
 [37.5439, -7.1842, 17.0],
 [37.5456, -7.1844, 17.0],
 [37.7942, -8.6233, 99.0],
 [37.7942, -8.6233, 199.0],
 [37.9017, -2.8986, 129.0],
 [37.9089, 13.4042, 41.0],
 [38.0944, -2.8089, 125.0],
 [38.0994, -7.04, 52.0],
 [38.3192, -8.5383, 54.0],
 [38.3214, -8.5353, 131.0],
 [38.4272, 16.1797, 20.0],
 [38.4311, -8.5353, 99.0],
 [38.4311, -8.5353, 199.0],
 [38.4736, -6.7578, 46.0],
 [38.4764, -2.4556, 130.0],
 [38.5197, -0.6314, 125.0],
 [38.6514, -8.6183, 17.0],
 [38.8461, 21.305, 46.0],
 [38.8744, 21.8658, 101.0],
 [38.9631, -8.075, 54.0],
 [39.3489, 8.5689, 46.0],
 [39.39, -4.1175, 43.0],
 [39.7856, 22.7939, 44.0],
 [39.7908, -8.9617, 130.0],
 [39.7978, 22.7772, 18.0],
 [39.8247, 2.9064, 46.0],
 [39.8544, 3.1814, 125.0],
 [40.075, -8.2544, 130.0],
 [40.1906, -6.4433, 54.0],

In [42]:
for ix, dfi in dfR_icp.iterrows():
    if list(dfi[['Lat', 'Lon', 'sp']].values) not in df_locs:
        dfR_icp = dfR_icp.drop(index=ix)
print(len(dfR_icp.groupby(['Lon', 'Lat', 'sp']).count().index))

996


In [43]:
df_all['sp'] = round(df_all['sp'], 0)
df_all = pd.merge(df_all, dfR_icp, on=['Lat', 'Lon', 'year', 'sp'], how='outer')
print(len(df_all.groupby(['Lon', 'Lat', 'sp']).count().index))
df_all

996


,year,CO2,Lat,Lon,n,ymin,ymax,count,yrange,sp,...,TC_pre_yi,TC_pre_gs_yi,TC_pre_gs_peak_yi,TC_tmp_yi,TC_tmp_gs_yi,TC_tmp_gs_peak_yi,TC_par_yi,TC_par_gs_yi,TC_par_gs_peak_yi,LNC
0,1990,0.000354,34.9294,32.9311,9.772500,2005,2011,4,6,121.0,...,0.001502,0.000643,0.000333,13.468182,17.766667,18.65,0.000445,0.000566,0.000633,NaN
1,1990,0.000354,34.9294,32.9311,8.696250,2005,2011,4,6,129.0,...,0.001502,0.000643,0.000333,13.468182,17.766667,18.65,0.000445,0.000566,0.000633,NaN
2,1990,0.000354,34.9522,32.8306,7.951250,2005,2011,4,6,129.0,...,0.001351,0.000583,0.000300,14.090909,18.233333,19.15,0.000452,0.000575,0.000641,NaN
3,1990,0.000354,35.0139,33.0500,11.116250,2005,2011,4,6,121.0,...,0.000958,0.000632,0.000433,17.254545,19.383333,14.45,0.000470,0.000578,0.000512,NaN
4,1990,0.000354,35.0200,32.6275,9.713750,2005,2011,4,6,121.0,...,0.001255,0.000562,0.000333,14.309091,18.283333,19.40,0.000454,0.000577,0.000640,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26912,2016,0.000404,67.9994,24.2361,9.700758,1995,2015,11,20,118.0,...,0.001705,0.003113,0.004194,0.466667,10.712500,14.80,0.000173,0.000273,0.000347,NaN
26913,2016,0.000404,68.9953,19.4275,12.715000,1995,1999,3,4,118.0,...,0.001479,0.002159,0.002613,2.654167,11.650000,13.95,0.000174,0.000323,0.000335,NaN
26914,2016,0.000404,69.4542,30.0361,11.912000,1995,2003,5,8,134.0,...,0.001578,0.002488,0.002387,1.625000,11.212500,14.85,0.000171,0.000285,0.000373,NaN
26915,2016,0.000404,69.5831,28.8981,12.803636,1995,2015,11,20,134.0,...,0.001449,0.002324,0.002355,1.354167,11.212500,15.00,0.000169,0.000277,0.000355,NaN


### all model data

In [44]:
df_all = df_all.groupby(['Lat', 'Lon', 'year', 'sp']).mean()
df_all.to_csv(os.path.join(data_dir, 'model_inputs', 'df_model_all_annual.csv'))

In [45]:
len(df_all.index)/(2016 - 1990+1)

996.0